In [2]:
# Clone the entire repo.
!git clone -l -s https://github.com/AdarshKumar712/Alice-in-Wonderland.git cloned-repo/
%cd cloned-repo
!ls

Cloning into 'cloned-repo'...
remote: Enumerating objects: 8, done.
remote: Total 8 (delta 0), reused 0 (delta 0), pack-reused 8
Unpacking objects: 100% (8/8), done.
/content/cloned-repo
README.md  SimpleRNN


In [3]:
cd SimpleRNN

/content/cloned-repo/SimpleRNN


In [0]:
# Importing the required modules
from __future__ import print_function
from keras.models import Sequential
from keras.layers.recurrent import SimpleRNN
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import Dropout
from keras.utils import vis_utils
import keras
import numpy as np

In [0]:
# Import the Dataset and preprocess
alice = open('alice.txt', 'rb')
lines = []
for line in alice:
    line = line.strip().lower()
    line = line.decode('ascii', 'ignore')
    if len(line)==0:
        continue
    lines.append(line)
alice.close()
text = " ".join(lines)

In [0]:
# Creating sets for comparison in RNN
## Identifying the characters and creating lookup dictionaries for them
chars = set([c for c in text])
len_chars = len(chars)
chars2index = dict([(c, i) for i, c in enumerate(chars)])
index2chars = dict([(i, c) for i, c in enumerate(chars)])


In [0]:
SEQ_LEN = 20
STEP = 1
input_chars = []
label_chars = []
for i in range(0, len(text)-SEQ_LEN, STEP):
    input_chars.append(text[i:i+SEQ_LEN])
    label_chars.append(text[i+SEQ_LEN])


In [0]:
# Vectorizing Labels
x = np.zeros((len(input_chars), SEQ_LEN, len_chars), dtype=np.bool)
y = np.zeros((len(input_chars), len_chars), dtype=np.bool)

for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        x[i, j, chars2index[ch]] = 1
    y[i, chars2index[label_chars[i]]] = 1

In [0]:
# HyperParameter Definition

HIDDEN_SIZE = 128
BATCH_SIZE = 128
NUM_ITERATIONS = 25
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100

# Creating the Model
model = Sequential()
model.add(Bidirectional(LSTM(128,return_sequences = True,input_shape=(SEQ_LEN, len_chars), unroll=True)))
model.add(Bidirectional(LSTM(128,return_sequences = True, unroll=True)))
model.add(LSTM(128,activation = 'relu',unroll = True,return_sequences = False))
model.add(Dense(len_chars, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])


In [12]:
# Predicting and testing the model
for iteration in range(NUM_ITERATIONS):
    print('='*50)
    print("Iteration #: %d"%(iteration))
    model.fit(x, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]
    print("Generating text from the seed : %s \n"%(test_chars))
    print(test_chars, end='')
    for i in range(NUM_PREDS_PER_EPOCH):
        X_test = np.zeros((1, SEQ_LEN, len_chars))
        for i, ch in enumerate(test_chars):
            X_test[0, i, chars2index[ch]] = 1
        pred = model.predict(X_test, verbose=False)[0]
        y_pred = index2chars[np.argmax(pred)]
        print(y_pred, end='')
        test_chars=test_chars[1:] + y_pred
print()

Iteration #: 0
Epoch 1/1
158763/158763 [==============================] - 172s 1ms/step - loss: 2.3942 - acc: 0.3162
Generating text from the seed : were seated on their 

were seated on their the the the the the the the the the the the the the the the the the the the the the the the the the==================================================
Iteration #: 1
Epoch 1/1
158763/158763 [==============================] - 145s 914us/step - loss: 1.8692 - acc: 0.4450
Generating text from the seed : id the king eagerly, 

id the king eagerly, and the mound the mound the mound the mound the mound the mound the mound the mound the mound the m==================================================
Iteration #: 2
Epoch 1/1
158763/158763 [==============================] - 145s 912us/step - loss: 1.6520 - acc: 0.5049
Generating text from the seed :  was, or longitude e 

 was, or longitude even the mouse to the mouse to the mouse to the mouse to the mouse to the mouse to the mouse to the m==================

In [13]:
for iteration in range(5):
    print('='*50)
    print("Iteration #: %d"%(iteration))
    model.fit(x, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]
    print("Generating text from the seed : %s \n"%(test_chars))
    print(test_chars, end='')
    for i in range(NUM_PREDS_PER_EPOCH):
        X_test = np.zeros((1, SEQ_LEN, len_chars))
        for i, ch in enumerate(test_chars):
            X_test[0, i, chars2index[ch]] = 1
        pred = model.predict(X_test, verbose=False)[0]
        y_pred = index2chars[np.argmax(pred)]
        print(y_pred, end='')
        test_chars=test_chars[1:] + y_pred
print()

Iteration #: 0
Epoch 1/1
158763/158763 [==============================] - 145s 912us/step - loss: 0.6070 - acc: 0.7995
Generating text from the seed :  a very interesting  

 a very interesting the table, and sometimes she could hear the matter where you are out of the work in a moment that sh==================================================
Iteration #: 1
Epoch 1/1
158763/158763 [==============================] - 145s 911us/step - loss: 0.5795 - acc: 0.8092
Generating text from the seed : reat hall, with the  

reat hall, with the bottle as she picked her so, said alice, a little snar offended togeing itself up and a poor and lit==================================================
Iteration #: 2
Epoch 1/1
158763/158763 [==============================] - 145s 912us/step - loss: 0.5520 - acc: 0.8175
Generating text from the seed : t slowly after it: i 

t slowly after it: i shall dinah are oftended tone, and a large plate came to the project gutenberg literary archive fou================

In [0]:
model.save('my_model.h5')

In [0]:
model_1 = keras.models.load_model('my_model.h5')

In [23]:
model_1.evaluate(x,y,verbose=1)

158763/158763 [==============================] - 208s 1ms/step


[0.4339650188220227, 0.8609688655432212]

In [22]:
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]
    print("Generating text from the seed : %s \n"%(test_chars))
    print(test_chars, end='')
    for i in range(1000):
        X_test = np.zeros((1, SEQ_LEN, len_chars))
        for i, ch in enumerate(test_chars):
            X_test[0, i, chars2index[ch]] = 1
        pred = model_1.predict(X_test, verbose=False)[0]
        y_pred = index2chars[np.argmax(pred)]
        print(y_pred, end='')
        test_chars=test_chars[1:] + y_pred
        if (y_pred == '.') or y_pred == ',' or y_pred==':':
          print()

Generating text from the seed :  foundation, the tra 

 foundation, the trademark owner,
 said the king.
 when she was to herself,
 as she went on against it:
 at last the little bittle bit stop in the subjects of te libus,
 still it had to late the table,
 and the words came upon a tree in any walk in the fand of great descoftly swam of dear! but those cause the mock turtle sang this:
 lasted their heads off? to look through the was of the baby?,
 im too much far and all the panther is,
 said the king.
 when she was to herself,
 as she went on against it:
 at last the little bittle bit stop in the subjects of te libus,
 still it had to late the table,
 and the words came upon a tree in any walk in the fand of great descoftly swam of dear! but those cause the mock turtle sang this:
 lasted their heads off? to look through the was of the baby?,
 im too much far and all the panther is,
 said the king.
 when she was to herself,
 as she went on against it:
 at last the little bittle bit st